In [2]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage
from langgraph.graph.message import add_messages
from typing import Annotated, List
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [5]:
load_dotenv()  # Load environment variables from .env file

True

In [6]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)

In [37]:
# Defining State
class JokeState(BaseModel):
    topic: Annotated[str, Field(description="The topic of the joke")]
    joke: Annotated[str, Field(description="The generated joke")]
    explanation: Annotated[str, Field(description="Explanation of why the joke is funny")]

In [38]:
# generate_joke function
def generate_joke(state: JokeState):
    prompt = f"Tell me a joke about {state.topic}"
    response = llm.invoke(prompt).content
    return {"joke": response}


In [30]:
# explain joke function
def explain_joke(state: JokeState):
    prompt = f"Explain the joke '{state.joke}'"
    response = llm.invoke(prompt).content
    return {"explanation": response}

In [39]:
# Checkpoiner
checkpointer = InMemorySaver()
# defining the graph
graph = StateGraph(JokeState)

# Adding nodes
graph.add_node("generate_joke", generate_joke)
graph.add_node("explain_joke", explain_joke)

# Defining edges
graph.add_edge(START, "generate_joke")
graph.add_edge("generate_joke", "explain_joke")
graph.add_edge("explain_joke", END)

# compiling the graph
workflow = graph.compile(checkpointer=checkpointer)


In [40]:
thread_id = "001"
config1 = {'configurable':{"thread_id": thread_id}}
initial_state = JokeState(topic="biriyani", joke="", explanation="")
workflow.invoke(initial_state, config=config1)

{'topic': 'biriyani',
 'joke': 'Why did the biriyani get invited to all the parties?\n\nBecause it had so many **layers** of personality, and it always brought the **spice**!',
 'explanation': 'This joke is a pun that plays on the literal characteristics of biryani and applies them metaphorically to a person\'s personality and social contribution.\n\nHere\'s the breakdown:\n\n1.  **Biryani\'s Literal Characteristics:**\n    *   **Layers:** Biryani is traditionally made by layering rice, meat/vegetables, fried onions, herbs, and spices. It\'s a dish built in distinct, flavorful layers.\n    *   **Spice:** Biryani is famous for its rich and complex flavor profile, achieved through a variety of aromatic spices (cardamom, cinnamon, cloves, chili, turmeric, etc.). It\'s a dish that definitely brings the "spice" in terms of taste.\n\n2.  **The Pun/Metaphorical Meaning:**\n    *   **"Layers of personality":** This refers to a person who is complex, interesting, deep, and multifaceted – not su

In [35]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'burger', 'joke': 'What did the burger say to the salad?\n"Lettuce be friends!"', 'explanation': 'This is a classic **pun** (a play on words) that works on two levels:\n\n1.  **Homophone:** The word "**lettuce**" sounds exactly like the phrase "**let us**."\n2.  **Contextual Relevance:** "Lettuce" is a common ingredient found in **both** burgers and salads, making it a fitting word for either food item to "say" in a conversation with the other.\n\nSo, the burger is essentially saying "Let us be friends!" but cleverly incorporating a food item that connects them both. The humor comes from the unexpected wordplay and the silly anthropomorphism of food.'}, next=(), config={'configurable': {'thread_id': '001', 'checkpoint_ns': '', 'checkpoint_id': '1f0c860a-52c7-65bc-8006-9d5f7854e712'}}, metadata={'source': 'loop', 'step': 6, 'parents': {}}, created_at='2025-11-23T11:36:26.205535+00:00', parent_config={'configurable': {'thread_id': '001', 'checkpoint_ns': ''

In [41]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'biriyani', 'joke': 'Why did the biriyani get invited to all the parties?\n\nBecause it had so many **layers** of personality, and it always brought the **spice**!', 'explanation': 'This joke is a pun that plays on the literal characteristics of biryani and applies them metaphorically to a person\'s personality and social contribution.\n\nHere\'s the breakdown:\n\n1.  **Biryani\'s Literal Characteristics:**\n    *   **Layers:** Biryani is traditionally made by layering rice, meat/vegetables, fried onions, herbs, and spices. It\'s a dish built in distinct, flavorful layers.\n    *   **Spice:** Biryani is famous for its rich and complex flavor profile, achieved through a variety of aromatic spices (cardamom, cinnamon, cloves, chili, turmeric, etc.). It\'s a dish that definitely brings the "spice" in terms of taste.\n\n2.  **The Pun/Metaphorical Meaning:**\n    *   **"Layers of personality":** This refers to a person who is complex, interesting, deep, and m

In [24]:
thread_id = "002"
config2 = {'configurable':{"thread_id": thread_id}}
initial_state = JokeState(topic="computer", joke="", explanation="")
workflow.invoke(initial_state, config=config2)

{'topic': 'computer',
 'joke': 'Why was the computer cold?\n\nBecause it left its Windows open!',
 'explanation': 'This is a classic pun! Here\'s the breakdown:\n\n1.  **Literal Meaning of "Windows":** In a house or building, if you leave the physical windows open, cold air comes in, making the room (and you) cold.\n\n2.  **Computer Meaning of "Windows":** "Windows" refers to the Microsoft Windows operating system, which is a very common operating system for personal computers. On a computer, you "open" and "close" various programs, folders, and web pages, which appear as rectangular "windows" on your screen.\n\n**The Joke:**\nThe joke plays on the double meaning of "Windows." It takes the literal reason someone might be cold (leaving physical windows open) and applies it to a computer, using the computer\'s "Windows" (the operating system and its graphical interface).\n\nThe humor comes from the absurdity of a computer feeling cold because of its software, but the wordplay makes it so

In [26]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza get a job?\n\nBecause it kneaded the dough!', 'explanation': 'This is a classic pun! Here\'s why it\'s funny:\n\n1.  **"Kneaded the dough" (literal meaning):** In the context of pizza, "kneading the dough" refers to the physical act of pressing and stretching the mixture of flour, water, yeast, etc., to prepare it for baking. This is a crucial step in making pizza.\n\n2.  **"Needed the dough" (pun meaning):**\n    *   "Kneaded" sounds exactly like "needed."\n    *   "Dough" is a common slang term for **money**.\n\nSo, the joke plays on the sound-alike words. The pizza got a job because it **needed money** (like a person), but the phrase is cleverly disguised using a literal action associated with pizza-making ("kneading the dough").\n\nThe humor comes from the unexpected human-like motivation (needing money) applied to an inanimate object (pizza), using a pun that ties back to its nature.'}, next=(), config={'configurab